# Replaying Hi-res Events to Test Controller

This is an example of replaying a last failure log back to a MAXTIME Emulator. First, use advance settings to set the port number when launching the emulator. The, you must set the UDP Port to match (admin/communication settings/advance settings).

Port 443 below is just an example, you can use different ports

In [1]:
from atc_replay import ATCReplay

ip_port = ('127.0.0.1', 443)
db_path = 'LastFailEventLog.db'
cycle_length = 100 # for timing when to start the simulation

# Your virtual conflict monitor :)
incompatible_pairs = [
  # Phases
  ('Ph1', 'Ph2'),
  ('Ph1', 'Ph4'),
  ('Ph1', 'Ph8'),
  ('Ph2', 'Ph4'),
  ('Ph2', 'Ph8'),
  ('Ph4', 'Ph5'),
  ('Ph4', 'Ph6'),
  ('Ph5', 'Ph6'),
  ('Ph5', 'Ph8'),
  ('Ph6', 'Ph8'),
  # Overlaps
  ('O1', 'Ph4'),
  ('O1', 'Ph8'),
  ('O2', 'Ph1'),
  ('O2', 'Ph4'),
  ('O2', 'Ph8'),
  ('O5', 'Ph4'),
  ('O5', 'Ph8'),
  ('O6', 'Ph4'),
  ('O6', 'Ph8'),
]

# Instantiate ATCReplay (loads event log and converts it to activation feed for NTICP)
replay = ATCReplay(event_log=db_path,
                   ip_port=ip_port,
                   incompatible_pairs=incompatible_pairs,
                   simulation_speed=1,
                   cycle_length=cycle_length)

# Print simulation run time
run_time = round(replay.activation_feed['sleep_time_cumulative'].max(),1)
minutes, seconds = divmod(run_time, 60)
print(f'Simulation run time: {minutes} minutes and {seconds} seconds')

replay.activation_feed.head()

Simulation run time: 6.0 minutes and 30.0 seconds


,TimeStamp,sleep_time,DeviceId,group_number,DetectorType,state_integer,sleep_time_cumulative
0,2024-10-03 09:02:42.100,0.0,0,3,Vehicle,128,0.00
1,2024-10-03 09:02:42.300,0.2,0,3,Vehicle,0,0.04
2,2024-10-03 09:02:43.700,1.4,0,4,Vehicle,64,0.32
3,2024-10-03 09:02:44.000,0.3,0,1,Vehicle,4,0.38
4,2024-10-03 09:02:44.100,0.1,0,6,Vehicle,2,0.40


In [3]:
# Rerun the trial until a conflict is found
replay_attempts = 3
for i in range(replay_attempts):
    print(f'ATCReplay attempt {i+1}/{replay_attempts}')
    replay.run()
    replay.get_output_data()
    conflict_df = replay.conflict_check()
    if not conflict_df.empty:
        print(conflict_df)
        # Save event log output to a csv file
        replay.output_data.to_csv('simulated_failure_log.csv', index=False)
        break

ATCReplay attempt 1/3
Waiting for 93.339346 seconds until the next cycle starts
                  TimeStamp    Conflict_Details
451 2024-10-10 12:29:57.100  O5 & Ph4; O5 & Ph8
